In [1]:
import tensorflow as tf
import os

print(tf.__version__)

2.0.0


In [2]:
class Mymodel(tf.keras.Model):
    def __init__(self):
        super().__init__()
        self.flatten = tf.keras.layers.Flatten()    # Flatten层将除第一维（batch_size）以外的维度展平
        self.dense1 = tf.keras.layers.Dense(units=256, activation=tf.nn.relu)
        self.dense2 = tf.keras.layers.Dense(units=10)

    def call(self, inputs):         
        x = self.flatten(inputs)   
        x = self.dense1(x)    
        output = self.dense2(x)     
        return output

In [3]:
X = tf.random.uniform((2,20))
net = Mymodel()
net(X)

<tf.Tensor: id=62, shape=(2, 10), dtype=float32, numpy=
array([[-0.4715509 , -0.1818477 , -0.33812922, -0.07651203,  0.13396965,
         0.16773072, -0.23122482, -0.13911861, -0.70685184, -0.1268108 ],
       [-0.41337287, -0.28741118, -0.38340008, -0.1244923 ,  0.22581719,
         0.12045028, -0.05472657, -0.2379032 , -0.7561487 , -0.17371078]],
      dtype=float32)>

In [4]:
# 构造一个复杂点的模型
class FancyMLP(tf.keras.Model):
    def __init__(self):
        super().__init__()
        self.flatten = tf.keras.layers.Flatten()
        self.rand_weight = tf.constant(
            tf.random.uniform((20,20)))
        self.dense = tf.keras.layers.Dense(units=20, activation=tf.nn.relu)

    def call(self, inputs):         
        x = self.flatten(inputs)   
        x = tf.nn.relu(tf.matmul(x, self.rand_weight) + 1)
        x = self.dense(x)
        x = tf.nn.relu(tf.matmul(x, self.rand_weight) + 1)
        x = self.dense(x)
        while tf.norm(x) > 1:
            x /= 2
        if tf.norm(x) < 0.8:
            x *= 10
        return tf.reduce_sum(x)


In [5]:
net = FancyMLP()
net(X)


<tf.Tensor: id=192, shape=(), dtype=float32, numpy=20.866703>

In [7]:
# 嵌套调用

class NestMLP(tf.keras.Model):
    def __init__(self):
        super().__init__()
        self.net = tf.keras.Sequential()
        self.net.add(tf.keras.layers.Flatten())
        self.net.add(tf.keras.layers.Dense(64, activation=tf.nn.relu))
        self.net.add(tf.keras.layers.Dense(32, activation=tf.nn.relu))
        self.dense = tf.keras.layers.Dense(units=16, activation=tf.nn.relu)


    def call(self, inputs):         
        return self.dense(self.net(inputs))

net = tf.keras.Sequential()
net.add(NestMLP())
net.add(tf.keras.layers.Dense(20))
net.add(FancyMLP())

net(X)


<tf.Tensor: id=422, shape=(), dtype=float32, numpy=21.893745>

In [8]:
net.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
nest_mlp (NestMLP)           multiple                  3952      
_________________________________________________________________
dense_6 (Dense)              multiple                  340       
_________________________________________________________________
fancy_mlp_1 (FancyMLP)       multiple                  420       
Total params: 4,712
Trainable params: 4,712
Non-trainable params: 0
_________________________________________________________________
